# Data Preprocessing for Stock Price Prediction

## Objectives
The goal of this notebook is to preprocess the stock price data to prepare it for analysis and modeling.
This will include:
1. Loading the data  
2. Handling missing data
3. Feature engineering and target creation
4. Scaling the data

## Importing Necessary Libraries
First, we need to import all the libraries required to run what we want to do.
We will be importing:
- **pandas**: A library for data manipulation and analysis. We will use it to load and clean the data, as well as to perform feature engineering
- **scikit-learn**: This library is a fundamental machine learning library. It allows to standardise the data.

In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

## 1. Load the Data
First we need to load the raw stock price data saved as a CSV file. This file should have already been created using data_collection.ipynb (KO_data.csv).

We will be doing this by creating a function called "load_data". It will:
- Read the CSV file into a Pandas DataFrame
- Remove any completely empty rows
- Reset the index to ensure proper ordering

In [14]:
def load_data(file_path):
    """
    Load the stock price data from a CSV file.
    Args:
        file_path (str): Path to the CSV file
    Returns:
        pd.DataFrame: Loaded data as a Pandas DataFrame
    """
    try:
        # Load the dataset from the CSV file
        data = pd.read_csv(file_path, thousands=',', skip_blank_lines=True)
       
        # Remove any empty rows
        data = data.dropna(how='all')
       
        # Reset the index to ensure continuity
        data = data.reset_index(drop=True)
        
        print("Data loaded successfully.")
        print(data)
        return data
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
    

## 2. Handle Missing Data
Next we will need to clean the dataset by handling any missing data and remove any rows that only display dividends.

Note: we will be using a rolling mean over a 7-day window to fill in any missing data.

In [15]:
def handle_missing_data(df):
    """
    Fill missing values using a rolling mena(7-day window) and remove "Dividend" rows.
    Args:
        df (pd.DataFrame): Dataframe containing stock prices.
    Returns:
        pd.DataFrame: Cleaned dataframe
    """
    # Remove rows where any column contains the string "Dividend"
    df = df[~df.apply(
        lambda row: row.astype(str).str.contains("Dividend", case=False, na=False).any(), axis=1
    )]
    
    # Fill missing values for all columns
    for column in df.columns: 
        if column != 'Date': # Skip the 'Date' column
            df.loc[:, column] = df[column].fillna(df[column].rolling(window=7, min_periods=1, center=True).mean())
    
    # Reset data types
    df['Open'] = df['Open'].astype(float)
    
    return df

## 3. Feature Engineering and Target Creation
After having the CSV file cleaned, we will now create new potential features which may be useful for the linear regression.
We will be engineering the following features:

### Moving Averages and Bollinger Bands
- **5-day Simple Moving Average (SMA_5)**: Average closing price over the last 5 days.
- **20-day Simple Moving Average (SMA_20)**: Average closing price over the last 20 days.
- **20-day Exponential Moving Average (EMA_20)**: Weighted average over the last 20 days but with more emphasis on recent data.
- **50-day Exponential Moving Average (EMA_50)**: Similar to EMA_20 but over 50 days.
- **Bollinger Bands (BB_upper, BB_lower)**: Indicate overbought or oversold conditions.

### Volume-Based Indicators
- **On-Balance Volume (OBV)**: Tracks volume flow.
- **Volume Weighted Average Price (VWAP)**: Average price adjusted for volume.

### Momentum Indicators
- **Relative Strength Index (RSI)**: Indicates whether the stock is overbought or oversold.
- **MACD**: Moving Average Convergence Divergence (MACD) shows trend strength.
- **Stochastic Oscillator (%K, %D)**: Indicates momentum based on the closing price relative to high/low range.


### Volatility Indicators
- **Historical Volatility**: Standard deviation of percentage price changes.
- **Rolling Standard Deviation**: Tracks price dispersion.

Finally, we define the **target variable (Price Movement)** as the difference between the closing and opening prices.

Note 1: We have engineered a lot of features which if all were used in our model may lead to overfitting.

Note 2: We have a lot of features we want to create, so to maintain a clear structure and some level of modularity we will be splitting the features into different categories and create each category as a function.


In [16]:
# 1. Moving Averages(SMA, EMA)
def add_moving_averages(df):
    """
    Add moving averages and Bollinger Bands.

    Args:
        df (pd.DataFrame): Dataframe containing stock prices.

    Returns:
        pd.DataFrame: Dataframe with new moving average features.
    """
    
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['BB_upper'] = df['SMA_20'] + (2 * df['Close'].rolling(window=20).std())
    df['BB_lower'] = df['SMA_20'] - (2 * df['Close'].rolling(window=20).std())
    return df
    
# 2. Volume-Based Indicators
def add_volume_indicators(df):
    """
    Add volume-based indicators (OBV, VWAP).

    Args:
        df (pd.DataFrame): Dataframe containing stock prices.

    Returns:
        pd.DataFrame: Dataframe with new volume-based features.
    """
    df['OBV'] = (df['Volume'] * ((df['Close'] - df['Close'].shift(1)) > 0).astype(int)).cumsum()
    df['VWAP'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum()
    return df

# 3. Momentum Indicators
def add_momentum_indicators(df):
    """
    Add Momentum indicators (RSI, MACD, Stochastic Oscillator)

    Args:
        df (pd.DataFrame): Dataframe containing stock prices.

    Returns:
        pd.DataFrame: Dataframe with new momentum features
    """
    #RSI calculations
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()                            
    rs = gain / (loss + 1e-10) # Included a miniscule value to the denominator to prevent dividing by 0.
    df['delta'] = delta
    df['rs'] = rs
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # MACD calculations
    ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # Stochastic Oscillator Calculations
    low_min = df['Low'].rolling(window=14).min()
    high_max = df['High'].rolling(window=14).max()
    df['%K'] = 100 * ((df['Close'] - low_min) / (high_max - low_min))
    df['%D'] = df['%K'].rolling(window=3).mean()
    
    return df

# 4. Volatility Indicators
def add_volatility_indicators(df):
    """
    Add volatility indicators (historical volatility, rolling std deviation).

    Args:
        df (pd.DataFrame): Dataframe containing stock prices.

    Returns:
        pd.DataFrame: Dataframe with new volatility features.
    """
    df['historical_volatility'] = df['Close'].pct_change().rolling(window=30).std()
    df['rolling_std'] = df['Close'].rolling(window=20).std()
    return df

# Apply all features
def apply_all_features(df):
    """
    Apply all feature engineering functions.

    Args:
        df (pd.DataFrame): Dataframe containing stock prices.

    Returns:
        pd.DataFrame: Dataframe with all new features.
    """
    df = add_moving_averages(df)
    df = add_volume_indicators(df)
    df = add_momentum_indicators(df)
    df = add_volatility_indicators(df)
    
    return df

# Create the target
def target_creation(df):
    """
    Create the target variable (price movement).

    Args:
        df (pd.DataFrame): Dataframe containing stock prices.

    Returns:
        pd.DataFrame: Dataframe with target variable.
    """
    df['Price Movement'] = df['Close'].to_numpy() - df['Open'].to_numpy()
    return df



## 4. Scaling the data
Finally, we need to scale the data to ensure all features have a similar range, aiding the performance and convergence speed of the linear regression model. We will be using the standardising way of scaling the data using the Z-formula 
(X - mean) / standard deviation.

Note: we will be putting the newly scaled data into new columns as to allow us to graph the scaled and unscaled features in exploratory_data_analysis.ipynb

In [17]:
def scale_data(df):
    scaler = StandardScaler()
    
    # Drop non-numerical columns and target column
    features = df.drop(['Date', 'Price Movement'], axis=1)
    
    # Apply scaling
    scaled_features = scaler.fit_transform(features)
    
    # Create a new DataFrame with the scaled features, adjusting their headers
    scaled_df = pd.DataFrame(scaled_features, columns=[col + '_standardised' for col in features.columns])
    
    # Align indices to avoid misalignment
    scaled_df.index = df.index
    
    # Concatenate the scaled features back to the original DataFrame
    df = pd.concat([df, scaled_df], axis=1)
    
    return df

Now we run everything together.

In [18]:
if __name__ == "__main__":
    df = load_data('data/KO_data.csv')
    df = handle_missing_data(df)
    df = apply_all_features(df)
    df = target_creation(df)
    df = scale_data(df)
    print(df.head()) # To view the processed DataFrame
    
    # Save the DataFrame to a CSV file
    file_path = 'data/preprocessed_KO_data.csv'
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")
    
    
    

Data loaded successfully.
             Date   Open   High    Low  Close  Adj Close      Volume
0    Dec 18, 2024  62.81  63.64  62.79  62.85      62.85  20920300.0
1    Dec 17, 2024  62.45  63.55  62.32  63.40      63.40  21902400.0
2    Dec 16, 2024  63.23  63.86  62.35  62.55      62.55  24559700.0
3    Dec 13, 2024  63.57  63.84  63.11  63.12      63.12  13429000.0
4    Dec 12, 2024  63.55  63.92  63.22  63.84      63.84  17542100.0
..            ...    ...    ...    ...    ...        ...         ...
251  Dec 26, 2023  58.06  58.71  58.06  58.56      56.82   6422500.0
252  Dec 22, 2023  58.12  58.46  58.02  58.32      56.59   9028500.0
253  Dec 21, 2023  57.85  58.07  57.47  57.99      56.27  11725100.0
254  Dec 20, 2023  58.50  58.67  57.57  57.61      55.90  17701000.0
255  Dec 19, 2023  59.00  59.20  58.64  58.83      57.09  14612200.0

[256 rows x 7 columns]
           Date   Open   High    Low  Close  Adj Close      Volume   SMA_5  \
0  Dec 18, 2024  62.81  63.64  62.79  62.85 

/var/folders/l9/zmdgypk569x7yf8ttvk8fsbw0000gp/T/ipykernel_29331/2016457706.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Open'] = df['Open'].astype(float)


## Conclusion
In this notebook we cleaned the raw historical stock price data for Coca-Cola (KO) and added new features and the target variable. We also scaled the all the features by standardisation using a built-in Scikit-Learn library that uses the Z-score formula.

In the next step of the project we will explore the data, analysing it to look at relationships and more.
